In [1]:
import xgboost as xgb
import scipy.stats as sps
import pandas as pd
import numpy as np
import warnings
import itertools
from scipy.sparse import hstack
import gc
from  xgboost import XGBClassifier
from  tqdm import tqdm
warnings.filterwarnings(action='ignore')

/home/shenweichen/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
seed = 1024
np.random.seed(seed)

path = './X_v2/'

In [3]:
train_off = pd.read_pickle(path+'offline_' + 'train' + '_x_23_26' + '.pkl.new')

In [4]:
train_1 = train_off[train_off.clickDay<26]
del train_off

In [5]:
train_1.shape

(7700704, 133)

In [6]:
test_off = pd.read_pickle(path+ 'offline_' + 'test' + '_x_27_27' + '.pkl.new')

In [4]:
feature_names = ['creativeID', 'userID',
       'positionID', 'connectionType', 'telecomsOperator', 'age', 'gender',
       'education', 'marriageStatus', 'haveBaby', 'ht_province',
       'rd_province', 'sitesetID', 'positionType', 'adID',
       'camgaignID', 'advertiserID', 'appID', 'appPlatform',
     # 'user_start_install_cate_0',
                 'user_start_install_cate_1',
       'user_start_install_cate_2', 'user_start_install_cate_3',
       'user_start_install_cate_4', 'user_start_install_cate_5',
                 
       'appCategory', 'trick', 'first_diff', 'last_diff', 'user_hist_install', 'clickHour',
        'user_adID_click_day', 'user_adID_click_hour',
       'user_camgaignID_click_day', 'user_camgaignID_click_hour',
       'user_appID_click_day', 'user_appID_click_hour',

                 
                 'user_sitesetID_click_day',
       'user_sitesetID_click_hour', 'user_click_day',
                 
        'positionID_cvr_smooth','creativeID_cvr','userID_cvr','adID_cvr','appID_cvr',
     'positionID_sum_count', 'creativeID_sum_count', 'appID_sum_count',
       'adID_sum_count', 'userID_sum_count',
        'user_ad_click_day_mean', 'user_ad_click_day_min',
       'user_camgaign_click_day_min', 'user_app_click_day_mean',
       'user_app_click_day_max', 'user_app_click_day_min',
       'user_site_click_day_mean', 'user_site_click_day_max',
       'user_site_click_day_min', 'user_click_day_mean', 'user_click_day_max','user_click_day_min',
        'advertiserID_user_clicks_age_cut_0',
       'advertiserID_user_clicks_age_cut_1',
       'advertiserID_user_clicks_age_cut_2',
       'advertiserID_user_clicks_age_cut_3',
       'advertiserID_user_clicks_age_cut_4',
       'advertiserID_user_clicks_age_cut_5',
       'advertiserID_user_clicks_age_cut_6',
      'advertiserID_user_clicks_age_cut_7',
       'advertiserID_user_clicks_gender_0',
       'advertiserID_user_clicks_gender_1',
       'advertiserID_user_clicks_gender_2',
       'advertiserID_user_clicks_education_0',
       'advertiserID_user_clicks_education_1',
       'advertiserID_user_clicks_education_2',
       'advertiserID_user_clicks_education_3',
       'advertiserID_user_clicks_education_4',
       'advertiserID_user_clicks_education_5',
       'advertiserID_user_clicks_education_6',
       'advertiserID_user_clicks_education_7',
       'advertiserID_user_clicks_marriageStatus_0',
       'advertiserID_user_clicks_marriageStatus_1',
       'advertiserID_user_clicks_marriageStatus_2',
       'advertiserID_user_clicks_marriageStatus_3',
        
       'appID_user_clicks_age_cut_0', 'appID_user_clicks_age_cut_1',
       'appID_user_clicks_age_cut_2', 'appID_user_clicks_age_cut_3',
       'appID_user_clicks_age_cut_4', 'appID_user_clicks_age_cut_5',
       'appID_user_clicks_age_cut_6', 'appID_user_clicks_age_cut_7',
       'appID_user_clicks_gender_0', 'appID_user_clicks_gender_1',
       'appID_user_clicks_gender_2', 'appID_user_clicks_education_0',
       'appID_user_clicks_education_1', 'appID_user_clicks_education_2',
       'appID_user_clicks_education_3', 'appID_user_clicks_education_4',
       'appID_user_clicks_education_5', 'appID_user_clicks_education_6',
       'appID_user_clicks_education_7', 'appID_user_clicks_marriageStatus_0',
       'appID_user_clicks_marriageStatus_1',
       'appID_user_clicks_marriageStatus_2',
       'appID_user_clicks_marriageStatus_3', 
                 'appID_user_clicks_haveBaby_0',
       'appID_user_clicks_haveBaby_1', 'appID_user_clicks_haveBaby_2',
       'appID_user_clicks_haveBaby_3', 'appID_user_clicks_haveBaby_4',
       'appID_user_clicks_haveBaby_5', 'appID_user_clicks_haveBaby_6',]


In [ ]:
len(feature_names)

112

In [ ]:
dtrain = xgb.DMatrix(train_1[feature_names].values, train_1['label'].values, feature_names=feature_names)
del train_1

In [ ]:
dval = xgb.DMatrix(test_off[feature_names].values, test_off['label'].values, feature_names=feature_names)
del test_off

In [ ]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [21]:
params={
    'max_depth':8,
    'nthread':25,
    'eta':0.1,
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'subsample':0.7,
    'colsample_bytree':0.5,
    'silent':1,
    'seed':1123,
    'min_child_weight':10
    #'scale_pos_weight':0.5
}

In [ ]:
#new feature
clf=xgb.train(params,dtrain,
              num_boost_round=2000,
              early_stopping_rounds=20,
              evals=watchlist,
              verbose_eval=10)

In [29]:
clf=xgb.train(params2,dtrain,
              num_boost_round=2000,
              early_stopping_rounds=20,
              evals=watchlist,
              verbose_eval=10)

[0]	train-logloss:0.607314	val-logloss:0.60768
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.2318	val-logloss:0.234009
[20]	train-logloss:0.136529	val-logloss:0.139858
[30]	train-logloss:0.108295	val-logloss:0.112692
[40]	train-logloss:0.099429	val-logloss:0.104641
[50]	train-logloss:0.096274	val-logloss:0.102194
[60]	train-logloss:0.094776	val-logloss:0.101313
[70]	train-logloss:0.093972	val-logloss:0.101031
[80]	train-logloss:0.093292	val-logloss:0.100871
[90]	train-logloss:0.092596	val-logloss:0.10078
[100]	train-logloss:0.091917	val-logloss:0.100736
[110]	train-logloss:0.091223	val-logloss:0.100718
[120]	train-logloss:0.090683	val-logloss:0.100666
[130]	train-logloss:0.090131	val-logloss:0.100668
[140]	train-logloss:0.089633	val-logloss:0.100678
Stopping. Best iteration:
[125]	train-logloss:0.090418	val-logloss:0.100658



In [ ]:
clf.save_model('230_round_xgb.hdf5')

In [5]:
test_on = pd.read_pickle(path+'online_' + 'test' + '_x_31_31' + '.pkl.new')

In [6]:
train_on = pd.read_pickle(path+'online_' + 'train' + '_x_26_29' + '.pkl.new')

In [6]:
train_on.shape

(11947463, 133)

In [7]:
dtrain = xgb.DMatrix(train_on[feature_names].values, train_on['label'].values, feature_names=feature_names)
del train_on

In [8]:
watchlist = [(dtrain, 'train'), (dtrain, 'val')]

In [9]:
params={
    'max_depth':8,
    'nthread':25,
    'eta':0.1,
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'subsample':0.7,
    'colsample_bytree':0.5,
    'silent':1,
    'seed':1123,
    'min_child_weight':10
    #'scale_pos_weight':0.5
}

In [10]:
clf=xgb.train(params,dtrain,
              num_boost_round=140,
              early_stopping_rounds=20,
              evals=watchlist,
              verbose_eval=10)

[0]	train-logloss:0.607573	val-logloss:0.607573
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.233847	val-logloss:0.233847
[20]	train-logloss:0.140372	val-logloss:0.140372
[30]	train-logloss:0.113379	val-logloss:0.113379
[40]	train-logloss:0.105017	val-logloss:0.105017
[50]	train-logloss:0.102279	val-logloss:0.102279
[60]	train-logloss:0.10108	val-logloss:0.10108
[70]	train-logloss:0.10047	val-logloss:0.10047
[80]	train-logloss:0.100018	val-logloss:0.100018
[90]	train-logloss:0.09958	val-logloss:0.09958
[100]	train-logloss:0.099315	val-logloss:0.099315
[110]	train-logloss:0.099059	val-logloss:0.099059
[120]	train-logloss:0.098804	val-logloss:0.098804
[130]	train-logloss:0.098563	val-logloss:0.098563


In [23]:
clf=xgb.train(params,dtrain,
              num_boost_round=140,
              early_stopping_rounds=20,
              evals=watchlist,
              verbose_eval=10)

[0]	train-logloss:0.607473	val-logloss:0.607473
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.233101	val-logloss:0.233101
[20]	train-logloss:0.139046	val-logloss:0.139046
[30]	train-logloss:0.111538	val-logloss:0.111538
[40]	train-logloss:0.103209	val-logloss:0.103209
[50]	train-logloss:0.100479	val-logloss:0.100479
[60]	train-logloss:0.099372	val-logloss:0.099372
[70]	train-logloss:0.098729	val-logloss:0.098729
[80]	train-logloss:0.098285	val-logloss:0.098285
[90]	train-logloss:0.097849	val-logloss:0.097849
[100]	train-logloss:0.097454	val-logloss:0.097454
[110]	train-logloss:0.097109	val-logloss:0.097109
[120]	train-logloss:0.096812	val-logloss:0.096812
[130]	train-logloss:0.096525	val-logloss:0.096525


In [15]:
test_on = pd.read_pickle(path+'online_' + 'test' + '_x_31_31' + '.pkl.new')
dtest=xgb.DMatrix(test_on[feature_names].values,feature_names=feature_names)
del test_on


In [16]:
res=clf.predict(dtest)
result = pd.read_csv('../result/demo_result.csv',index_col=['instanceID'])
result['prob'] = res


In [17]:
result.describe()

,prob
count,3.321748e+06
mean,2.496050e-02
std,3.820211e-02
min,1.187740e-05
25%,2.172654e-03
50%,1.228836e-02
75%,3.310529e-02
max,8.703234e-01


In [18]:
result = pd.read_csv('./res/submission_6_23_depth_10.csv')

In [15]:
clf

In [11]:
print('train end!')

train end!


In [35]:
logloss(valid_y,pred_y)

0.094901951478717023

In [12]:
test_on = pd.read_pickle(path+'online_' + 'test' + '_x_31_31' + '.pkl.new')

In [13]:
dtest=xgb.DMatrix(test_on[feature_names].values,feature_names=feature_names)
del test_on

In [14]:
res=clf.predict(dtest)

In [16]:
result.describe()

,prob
count,3.321748e+06
mean,2.470205e-02
std,4.125664e-02
min,2.668653e-06
25%,1.627459e-03
50%,1.028007e-02
75%,3.171212e-02
max,9.019705e-01


In [15]:
result = pd.read_csv('../result/demo_result.csv',index_col=['instanceID'])
result['prob'] = res
result.to_csv('./res/submission.csv')

In [16]:
def inverse_logit(x):
    return np.log(x/(1-x))
def logit(x):
    return 1/(1+np.exp(-x))
def calibration(pred,avg):
    intercept = inverse_logit(np.mean(pred))- inverse_logit(avg)
    return logit(inverse_logit(pred) - intercept)

In [28]:
res = calibration(res,0.027232030146226993)

In [29]:
result = pd.read_csv('../result/demo_result.csv',index_col=['instanceID'])
result['prob'] = res

In [30]:
result.to_csv('./res/submission_6_23_depth_10_adj.csv')

In [1]:
import xgboost as xgb
import scipy.stats as sps
import pandas as pd
import numpy as np
import warnings
import itertools
from scipy.sparse import hstack
import gc
from  tqdm import tqdm
from  xgboost import XGBClassifier
from sklearn.model_selection import KFold

/home/shenweichen/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
warnings.filterwarnings(action='ignore')

seed = 1024
np.random.seed(seed)


path = './X_v2/'

In [3]:
train_25 = pd.read_pickle(path + 'online_' + 'train' + '_x_25_25' + '.pkl.6.26')

In [5]:
train =pd.read_pickle(path + 'online_' + 'train' + '_x_26_29' + '.pkl.6.26')
test = pd.read_pickle(path+'online_' + 'test' + '_x_31_31' + '.pkl.6.26')
feature_names = [#'creativeID', 'userID',
                 'positionID', 'connectionType', 'telecomsOperator', 'age', 'gender',
                 'education', 'marriageStatus', 'haveBaby', 'ht_province',
                 'rd_province', 'sitesetID', 'positionType', 'adID',
                 'camgaignID', 'advertiserID', 'appID', 'appPlatform',
                 # 'user_start_install_cate_0',
                 'user_start_install_cate_1',
                 'user_start_install_cate_2', 'user_start_install_cate_3',
                 'user_start_install_cate_4', 'user_start_install_cate_5',

                 'appCategory', 'trick', 'first_diff', 'last_diff', 'user_hist_install', 'clickHour',
                 'user_adID_click_day', 'user_adID_click_hour',
                 'user_camgaignID_click_day', 'user_camgaignID_click_hour',
                 'user_appID_click_day', 'user_appID_click_hour',

                 'user_sitesetID_click_day',
                 'user_sitesetID_click_hour', 'user_click_day',

                 'positionID_cvr_smooth', 'creativeID_cvr', 'userID_cvr', 'adID_cvr', 'appID_cvr',
                 'positionID_sum_count', 'creativeID_sum_count', 'appID_sum_count',
                 'adID_sum_count', 'userID_sum_count',
                 'user_ad_click_day_mean', 'user_ad_click_day_min',
                 'user_camgaign_click_day_min', 'user_app_click_day_mean',
                 'user_app_click_day_max', 'user_app_click_day_min',
                 'user_site_click_day_mean', 'user_site_click_day_max',
                 'user_site_click_day_min', 'user_click_day_mean', 'user_click_day_max', 'user_click_day_min',
                 'advertiserID_user_clicks_age_cut_0',
                 'advertiserID_user_clicks_age_cut_1',
                 'advertiserID_user_clicks_age_cut_2',
                 'advertiserID_user_clicks_age_cut_3',
                 'advertiserID_user_clicks_age_cut_4',
                 'advertiserID_user_clicks_age_cut_5',
                 'advertiserID_user_clicks_age_cut_6',
                 'advertiserID_user_clicks_age_cut_7',
                 'advertiserID_user_clicks_gender_0',
                 'advertiserID_user_clicks_gender_1',
                 'advertiserID_user_clicks_gender_2',
                 'advertiserID_user_clicks_education_0',
                 'advertiserID_user_clicks_education_1',
                 'advertiserID_user_clicks_education_2',
                 'advertiserID_user_clicks_education_3',
                 'advertiserID_user_clicks_education_4',
                 'advertiserID_user_clicks_education_5',
                 'advertiserID_user_clicks_education_6',
                 'advertiserID_user_clicks_education_7',
                 'advertiserID_user_clicks_marriageStatus_0',
                 'advertiserID_user_clicks_marriageStatus_1',
                 'advertiserID_user_clicks_marriageStatus_2',
                 'advertiserID_user_clicks_marriageStatus_3',

                 'appID_user_clicks_age_cut_0', 'appID_user_clicks_age_cut_1',
                 'appID_user_clicks_age_cut_2', 'appID_user_clicks_age_cut_3',
                 'appID_user_clicks_age_cut_4', 'appID_user_clicks_age_cut_5',
                 'appID_user_clicks_age_cut_6', 'appID_user_clicks_age_cut_7',
                 'appID_user_clicks_gender_0', 'appID_user_clicks_gender_1',
                 'appID_user_clicks_gender_2', 'appID_user_clicks_education_0',
                 'appID_user_clicks_education_1', 'appID_user_clicks_education_2',
                 'appID_user_clicks_education_3', 'appID_user_clicks_education_4',
                 'appID_user_clicks_education_5', 'appID_user_clicks_education_6',
                 'appID_user_clicks_education_7', 'appID_user_clicks_marriageStatus_0',
                 'appID_user_clicks_marriageStatus_1',
                 'appID_user_clicks_marriageStatus_2',
                 'appID_user_clicks_marriageStatus_3',
                 'appID_user_clicks_haveBaby_0',
                 'appID_user_clicks_haveBaby_1', 'appID_user_clicks_haveBaby_2',
                 'appID_user_clicks_haveBaby_3', 'appID_user_clicks_haveBaby_4',
                 'appID_user_clicks_haveBaby_5', 'appID_user_clicks_haveBaby_6','install2click']

In [8]:
train = pd.concat([train_25,train])

In [10]:
train.shape

(14597343, 134)

In [11]:
X_train = train[feature_names].values
X_test = test[feature_names].values
y = train['label'].values
# y_test = test['label'].values
del train,test
print(X_train.shape,X_test.shape)

(14597343, 113) (3321748, 113)


In [12]:
params={
    'max_depth':10,
    'nthread':25,
    'eta':0.1,
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'subsample':0.8,
    'colsample_bytree':0.7,
    'silent':1,
    'seed':1123,
    'min_child_weight':10
    #'scale_pos_weight':0.5
}

In [ ]:
#offline
dtrain = xgb.DMatrix(X_train, y)
dval = xgb.DMatrix(X_test, y_test)
watchlist = [(dtrain, 'train'), (dval, 'val')]
clf = xgb.train(params, dtrain,
                num_boost_round=300,
                early_stopping_rounds=20,
                evals=watchlist,
                verbose_eval=10)

[0]	train-logloss:0.607362	val-logloss:0.607858
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.232351	val-logloss:0.234143
[20]	train-logloss:0.137999	val-logloss:0.140361
[30]	train-logloss:0.110433	val-logloss:0.113139
[40]	train-logloss:0.102146	val-logloss:0.105129
[50]	train-logloss:0.099278	val-logloss:0.102525
[60]	train-logloss:0.098175	val-logloss:0.101633
[70]	train-logloss:0.097559	val-logloss:0.101193
[80]	train-logloss:0.097037	val-logloss:0.100892
[90]	train-logloss:0.096573	val-logloss:0.100694
[100]	train-logloss:0.096139	val-logloss:0.100497
[110]	train-logloss:0.095833	val-logloss:0.100414
[120]	train-logloss:0.095541	val-logloss:0.100335
[130]	train-logloss:0.095249	val-logloss:0.100241
[140]	train-logloss:0.094991	val-logloss:0.100185
[150]	train-logloss:0.094723	val-logloss:0.100141
[160]	train-logloss:0.094495	val-logloss:0.100096
[170]	train-lo

In [13]:
#online
dtrain = xgb.DMatrix(X_train, y)
dval = xgb.DMatrix(X_train, y)
watchlist = [(dtrain, 'train'), (dval, 'val')]
#1,1*best_round

In [10]:
clf = xgb.train(params, dtrain,
                num_boost_round=270,
                early_stopping_rounds=20,
                evals=watchlist,
                verbose_eval=10)

[0]	train-logloss:0.607498	val-logloss:0.607498
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.23315	val-logloss:0.23315
[20]	train-logloss:0.139064	val-logloss:0.139064
[30]	train-logloss:0.111693	val-logloss:0.111693
[40]	train-logloss:0.103244	val-logloss:0.103244
[50]	train-logloss:0.10045	val-logloss:0.10045
[60]	train-logloss:0.09932	val-logloss:0.09932
[70]	train-logloss:0.098716	val-logloss:0.098716
[80]	train-logloss:0.098236	val-logloss:0.098236
[90]	train-logloss:0.097842	val-logloss:0.097842
[100]	train-logloss:0.097428	val-logloss:0.097428
[110]	train-logloss:0.097109	val-logloss:0.097109
[120]	train-logloss:0.096811	val-logloss:0.096811
[130]	train-logloss:0.096536	val-logloss:0.096536
[140]	train-logloss:0.096279	val-logloss:0.096279
[150]	train-logloss:0.096078	val-logloss:0.096078
[160]	train-logloss:0.09588	val-logloss:0.09588
[170]	train-logloss:0.

In [14]:
clf = xgb.train(params, dtrain,
                num_boost_round=350,
                early_stopping_rounds=20,
                evals=watchlist,
                verbose_eval=10)

[0]	train-logloss:0.607544	val-logloss:0.607544
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.233485	val-logloss:0.233485
[20]	train-logloss:0.139313	val-logloss:0.139313
[30]	train-logloss:0.111852	val-logloss:0.111852
[40]	train-logloss:0.103623	val-logloss:0.103623
[50]	train-logloss:0.100859	val-logloss:0.100859
[60]	train-logloss:0.099772	val-logloss:0.099772
[70]	train-logloss:0.099156	val-logloss:0.099156
[80]	train-logloss:0.098699	val-logloss:0.098699
[90]	train-logloss:0.098356	val-logloss:0.098356
[100]	train-logloss:0.097936	val-logloss:0.097936
[110]	train-logloss:0.097595	val-logloss:0.097595
[120]	train-logloss:0.097307	val-logloss:0.097307
[130]	train-logloss:0.097083	val-logloss:0.097083
[140]	train-logloss:0.096871	val-logloss:0.096871
[150]	train-logloss:0.096679	val-logloss:0.096679
[160]	train-logloss:0.096451	val-logloss:0.096451
[170]	train-lo

In [15]:
dtest = xgb.DMatrix(X_test)
res = clf.predict(dtest)
print(np.mean(res))

0.024987


In [16]:
result = pd.read_csv('../result/demo_result.csv',index_col=['instanceID'])
result['prob'] = res
# result['prob'] = result['prob'].apply(adj)

result.to_csv('./res/xgb_6_28_iter_350.csv')

In [19]:
np.mean(res)

0.024987027

In [22]:
np.mean(res)

0.024987027

In [21]:
result.describe()

,prob
count,3.321748e+06
mean,2.488538e-02
std,4.141779e-02
min,5.996474e-07
25%,1.598774e-03
50%,1.063640e-02
75%,3.190567e-02
max,8.918992e-01


In [13]:
result.to_csv('./res/xgb_6_27_submission_4day.csv')

In [7]:
skf = KFold(n_splits=5, shuffle=True, random_state=1123).split(y)
dtest = xgb.DMatrix(X_test)


In [8]:
fold = 0
for ind_tr, ind_te in skf:
    X_ot_train = X_train[ind_tr]
    X_ot_test=X_train[ind_te]
    y_train = y[ind_tr]
    y_test = y[ind_te]
    dtrain = xgb.DMatrix(X_ot_train, y_train)
    dval = xgb.DMatrix(X_ot_test, y_test)
    watchlist = [(dtrain, 'train'), (dval, 'val')]
    clf = xgb.train(params, dtrain,
                    num_boost_round=180,
                    early_stopping_rounds=20,
                    evals=watchlist,
                    verbose_eval=10)
    del dtrain,dval
    tmp_res = clf.predict(dtest)
    pd.to_pickle(tmp_res,'./res/xgb_fold{0}.pkl'.format(fold))
    print(np.mean(tmp_res))
    res+=tmp_res
    fold = fold+1
res/=5.0

[0]	train-logloss:0.607501	val-logloss:0.607488
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.23324	val-logloss:0.233214
[20]	train-logloss:0.138959	val-logloss:0.139042
[30]	train-logloss:0.111559	val-logloss:0.111834
[40]	train-logloss:0.103094	val-logloss:0.103587
[50]	train-logloss:0.100345	val-logloss:0.101041
[60]	train-logloss:0.099222	val-logloss:0.100109
[70]	train-logloss:0.098565	val-logloss:0.099631
[80]	train-logloss:0.0981	val-logloss:0.099331
[90]	train-logloss:0.09765	val-logloss:0.099118
[100]	train-logloss:0.097268	val-logloss:0.098951
[110]	train-logloss:0.096929	val-logloss:0.098801
[120]	train-logloss:0.096592	val-logloss:0.098704
[130]	train-logloss:0.096334	val-logloss:0.098631
[140]	train-logloss:0.096036	val-logloss:0.098558
[150]	train-logloss:0.095848	val-logloss:0.098524
[160]	train-logloss:0.095623	val-logloss:0.09847
[170]	train-logloss

NameError: name 'res' is not defined

In [30]:
res = 0
for f in range(5):
    tmp_res=pd.read_pickle('./res/xgb_fold{0}.pkl'.format(f))
    print(np.mean(tmp_res))
    res+=tmp_res
res/=5.0
print(np.mean(res))

0.0250432
0.0249957
0.025126
0.025098
0.025017
0.025056


In [31]:
res

array([ 0.02392305,  0.00148875,  0.00064256, ...,  0.05364005,
        0.00725566,  0.00102117], dtype=float32)

In [14]:
def inverse_logit(x):
    return np.log(x/(1-x))
def logit(x):
    return 1/(1+np.exp(-x))
def calibration(pred,avg):
    intercept = inverse_logit(np.mean(pred))- inverse_logit(avg)
    return logit(inverse_logit(pred) - intercept)

In [15]:
res = calibration(res,0.027232030146226993)

In [16]:
result = pd.read_csv('../result/demo_result.csv',index_col=['instanceID'])
result['prob'] = res
# result['prob'] = result['prob'].apply(adj)

result.to_csv('./res/xgb_6_27_calibation.csv')

In [17]:
result.describe()

,prob
count,3.321748e+06
mean,2.695235e-02
std,4.346146e-02
min,1.464321e-06
25%,1.833458e-03
50%,1.179965e-02
75%,3.520554e-02
max,8.890951e-01


In [33]:
res = calibration(res,0.027232030146226993)
result = pd.read_csv('../result/demo_result.csv',index_col=['instanceID'])
result['prob'] = res
# result['prob'] = result['prob'].apply(adj)

result.to_csv('./res/xgb_6_25_submission_adj.csv')

In [34]:
result.describe()

,prob
count,3.321748e+06
mean,2.698495e-02
std,4.054799e-02
min,1.120272e-05
25%,2.325138e-03
50%,1.317475e-02
75%,3.594951e-02
max,8.540689e-01
